In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
race_infos = pd.read_csv('race_infos.csv',encoding="shift jis",index_col=0)
race_results = pd.read_csv('race_results.csv',encoding="shift jis",index_col=0)

In [6]:
race_infos.shape

(40407, 8)

In [4]:
race_results

,race_id,着順,枠番,馬番,馬名,斤量,騎手,タイム,上り,単勝,人気,性,年齢,体重,体重変化
0,200801010101,1,4,6,ディアジーナ,54.0,四位洋文,1:31.0,35.6,4.3,2,牝,2,468,0
1,200801010101,2,8,14,アイディンパワー,54.0,藤田伸二,1:31.4,35.9,5.2,4,牡,2,460,0
2,200801010101,3,1,1,ウインリーズン,54.0,安藤勝己,1:31.5,36.4,3.1,1,牡,2,442,2
3,200801010101,4,7,11,ヴァンエボン,54.0,小林徹弥,1:31.6,35.5,23.0,8,牡,2,482,-4
4,200801010101,5,2,2,ラヴェリータ,54.0,長谷川浩,1:31.6,35.7,21.9,6,牝,2,484,-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,202110040812,7,2,2,ジオルティ,51.0,角田大和,2:42.0,35.3,49.6,10,牡,3,502,8
7,202110040812,8,7,9,ロックグラス,57.0,幸英明,2:42.2,35.8,43.7,8,牡,4,464,0
8,202110040812,9,7,8,サマーカナロア,53.0,富田暁,2:42.2,35.4,46.5,9,牡,3,502,-10
9,202110040812,10,8,10,レッドレイル,57.0,浜中俊,2:42.4,36.3,14.0,6,セ,5,436,2


In [5]:
#dfを結合する
race_result_all = pd.merge(race_infos, race_results)
race_result_all.head(5)

,race_id,race_name,course_len,rotation,weather,race_type,ground_state,date,着順,枠番,...,斤量,騎手,タイム,上り,単勝,人気,性,年齢,体重,体重変化
0,200801010101,2歳未勝利,1500,右,曇,芝,良,2008年8月16日,1,4,...,54.0,四位洋文,1:31.0,35.6,4.3,2,牝,2,468,0
1,200801010101,2歳未勝利,1500,右,曇,芝,良,2008年8月16日,2,8,...,54.0,藤田伸二,1:31.4,35.9,5.2,4,牡,2,460,0
2,200801010101,2歳未勝利,1500,右,曇,芝,良,2008年8月16日,3,1,...,54.0,安藤勝己,1:31.5,36.4,3.1,1,牡,2,442,2
3,200801010101,2歳未勝利,1500,右,曇,芝,良,2008年8月16日,4,7,...,54.0,小林徹弥,1:31.6,35.5,23.0,8,牡,2,482,-4
4,200801010101,2歳未勝利,1500,右,曇,芝,良,2008年8月16日,5,2,...,54.0,長谷川浩,1:31.6,35.7,21.9,6,牝,2,484,-20


In [6]:
race_result_all.shape

(577474, 22)

In [7]:
first = race_result_all[race_result_all['着順']== 1]

In [8]:
first.columns

Index(['race_id', 'race_name', 'course_len', 'rotation', 'weather',
       'race_type', 'ground_state', 'date', '着順', '枠番', '馬番', '馬名', '斤量', '騎手',
       'タイム', '上り', '単勝', '人気', '性', '年齢', '体重', '体重変化'],
      dtype='object')

In [9]:
first_ = first[['着順','単勝','人気','race_id','race_name','course_len','weather','race_type']]

In [10]:
magnification_tmp=[]

for i in first_['単勝'].values:
    i = math.floor(i)
    magnification_tmp.append(i)

In [11]:
first_['倍率切捨'] = magnification_tmp

<ipython-input-11-2d298450a3f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_['倍率切捨'] = magnification_tmp


In [12]:
first_ = first_[['着順','単勝','倍率切捨','人気','race_id','race_name','course_len','weather','race_type']]

In [13]:
first_

,着順,単勝,倍率切捨,人気,race_id,race_name,course_len,weather,race_type
0,1,4.3,4,2,200801010101,2歳未勝利,1500,曇,芝
14,1,3.7,3,2,200801010102,3歳未勝利,1000,曇,ダート
25,1,2.1,2,1,200801010103,3歳未勝利,1700,曇,ダート
38,1,3.6,3,2,200801010104,2歳新馬,1500,曇,芝
52,1,12.1,12,5,200801010105,3歳未勝利,1700,曇,ダート
...,...,...,...,...,...,...,...,...,...
577414,1,26.1,26,8,202110040808,3歳以上1勝クラス,1000,曇,ダート
577426,1,1.7,1,1,202110040809,筑後川特別,1800,小雨,芝
577437,1,6.8,6,2,202110040810,西日本スポーツ杯,1700,小雨,ダート
577453,1,6.4,6,4,202110040811,第41回小倉2歳ステークス(G3),1200,小雨,芝


In [14]:
first_['race_name'].unique()

array(['2歳未勝利', '3歳未勝利', '2歳新馬', ..., '第57回小倉記念(G3)',
       '第56回TV西日本北九州記念(G3)', '第41回小倉2歳ステークス(G3)'], dtype=object)

In [15]:
popularity_count = pd.DataFrame(first_['人気'].value_counts()).reset_index()
popularity_count.rename(columns={'index':'人気', '人気':'count'}, inplace=True)
popularity_count['ratio'] = popularity_count['count'] / popularity_count['count'].sum()
popularity_count['cumulative_ratio'] = popularity_count['ratio'].cumsum()
popularity_count

,人気,count,ratio,cumulative_ratio
0,1,12971,0.320604,0.320604
1,2,7653,0.189159,0.509763
2,3,5330,0.131742,0.641505
3,4,3824,0.094518,0.736023
4,5,2888,0.071383,0.807405
5,6,2255,0.055737,0.863142
6,7,1630,0.040289,0.903431
7,8,1161,0.028696,0.932127
8,9,855,0.021133,0.953260
9,10,650,0.016066,0.969326


In [16]:
magnification_count = pd.DataFrame(first_['倍率切捨'].value_counts()).reset_index()
magnification_count.rename(columns={'index':'倍率切り捨て', '倍率切捨':'count'}, inplace=True)
magnification_count['ratio'] = magnification_count['count'] / magnification_count['count'].sum()
magnification_count['cumulative_ratio'] = magnification_count['ratio'].cumsum()
magnification_count

,倍率切り捨て,count,ratio,cumulative_ratio
0,2,6186,0.152899,0.152899
1,3,5583,0.137995,0.290894
2,1,4359,0.107741,0.398636
3,4,4182,0.103366,0.502002
4,5,3103,0.076697,0.578699
...,...,...,...,...
237,344,1,0.000025,0.999901
238,312,1,0.000025,0.999926
239,144,1,0.000025,0.999951
240,325,1,0.000025,0.999975


In [17]:
magnification_count_20 = magnification_count.iloc[:20, :]

In [18]:
magnification_count_20

,倍率切り捨て,count,ratio,cumulative_ratio
0,2,6186,0.152899,0.152899
1,3,5583,0.137995,0.290894
2,1,4359,0.107741,0.398636
3,4,4182,0.103366,0.502002
4,5,3103,0.076697,0.578699
5,6,2403,0.059395,0.638094
6,7,1946,0.048099,0.686193
7,8,1438,0.035543,0.721736
8,9,1175,0.029042,0.750779
9,10,931,0.023012,0.773790


In [52]:
%matplotlib tk

fig, ax1 = plt.subplots()
first_['倍率切捨'].value_counts().plot.bar(figsize=(15, 3), color='darkblue', ax=ax1)
ax2 = ax1.twinx()
ratio = first_['倍率切捨'].value_counts()/ first_['倍率切捨'].value_counts().sum()
ratio.cumsum().plot(figsize=(15, 3), color='lightblue', ax=ax2)

<AxesSubplot:label='44aecf8a-a125-449b-a8b7-09b2fd4c57ea'>